# Tarefa 03 de Álgebra Linear Computacional

Atividade sobre Decomposição LU e resolução de sistemas lineares.

* **Aluna:** Bárbara Neves
* **Matrícula:** 507526

### Descrição

Implementar a **Decomposição LU** de uma matriz e utilizá-la na solução de um sistema linear.

> **Nota:** 
Às vezes a matriz original não tem decomposição LU ou tem uma decomposição PLU onde $P$ é uma matriz de permutação. Vamos considerar apenas os casos em que a matriz tem decomposição LU sem recorrer à permutação de linhas.

# Imports

In [1]:
import numpy as np
from scipy.linalg import lu as scipy_lu

# Decomposição LU 

A **Decomposição LU** de uma matriz trata-se da fatoração de uma matriz quadrada em duas matrizes triangulares: uma matriz triangular superior e uma matriz triangular inferior, de modo que o produto dessas duas matrizes resulte na matriz original.

Isto é, dada uma matriz quadrada $A$, procuraremos as matrizes $L$ e $U$ tais que

- $A = LU$
- $L$ e $U$ são matizes quadradas.
- $L$ é uma matriz triangular inferior com entradas diagonais principais iguais a 1.
- $U$ é uma matriz triangular superior formada como resultado da aplicação do método de Eliminação de Gauss em $A$.

Abaixo está uma visualização do que estamos procurando.

\\

> Para $A = \begin{bmatrix}
a_{11} & a_{12} & a_{13} \\
a_{21} & a_{22} & a_{23} \\
a_{31} & a_{32} & a_{33} \\
\end{bmatrix}$, temos $L = \begin{bmatrix}
1 & 0 & 0 \\
l_{21} & 1 & 0 \\
l_{31} & l_{32} & 1 \\
\end{bmatrix}$ e $U = \begin{bmatrix}u_{11} & u_{12} & u_{13} \\0 & u_{22} & u_{23} \\0 & 0 & u_{33} \\\end{bmatrix}$; de modo que $A = LU$.

\\

---

\\

### Porque é a Decomposição LU é útil?

Suponha que encontramos $L$ e $U$ de modo que $A = LU$ e desejamos resolver o sistema $Ax = b$. Outra maneira de escrever o problema pode ser $LUx = B$. Podemos então definir um $y$ desconhecido dizendo que $Ux = y$, e trocar um único sistema $Ax = b$ por dois sistemas seguintes.

\\

\begin{gather*} 
  Ux = y \\
  Ly = b
\end{gather*}

\\

Apesar de ter o dobro do número de equações, os dois sistemas que são triangulares e podem ser resolvidos facilmente com *Back* ou *Foward Substitution*.

## Decomposição LU Simples

In [2]:
# Funções utilitárias
def is_square(A): 
  return all(len(row) == len(A) for row in A)

def forward_substitution(L, b):
    n = L.shape[0]
    
    y = np.zeros_like(b, dtype=np.double);
    y[0] = b[0] / L[0, 0]
    
    for i in range(1, n):
      y[i] = (b[i] - np.dot(L[i, :i], y[:i])) / L[i, i]
        
    return y

def back_substitution(U, y):
    n = U.shape[0]
    
    x = np.zeros_like(y, dtype=np.double);
    x[-1] = y[-1] / U[-1, -1]
    
    for i in range(n - 2, -1, -1):
      x[i] = (y[i] - np.dot(U[i, i:], x[i:])) / U[i, i]
        
    return x

In [3]:
def lu(A):
  n = A.shape[0]
  
  U = A.copy()
  L = np.eye(n, dtype=np.double)

  if not is_square(U):
    raise ValueError('Argumento inválido: a matriz de entrada deve ser quadrada.')

  if any(np.diag(U) == 0):
    raise ZeroDivisionError(('Pivot não suportado! Divisão por zero encontrada.'))
  
  for i in range(n):
    factor = U[i + 1:, i] / U[i, i]
    L[i + 1:, i] = factor
    U[i + 1:] -= factor[:, np.newaxis] * U[i]
      
  return L, U

In [4]:
def lu_solve(A, b):
    L, U = lu(A)
    assert L.shape == U.shape
    
    y = forward_substitution(L, b)
    return back_substitution(U, y)

### Exemplo 1

\begin{gather*} 
  x_1 + x_2 + x_3 = 1 \\
  4x_1 + 3x_2 - x_3 = 6 \\
  3x_1 + 5x_2 + 3x_3 = 4
\end{gather*}

Assim, 

\begin{gather*} 
  A_1 = \begin{bmatrix}
  1 & 1 & 1 \\ 
  4 & 3 & -1 \\ 
  3 & 5 & 3
  \end{bmatrix} 
  \qquad
  b_1 = \begin{bmatrix}
  1 \\ 
  6 \\ 
  4 
  \end{bmatrix}
\end{gather*}

In [6]:
A_1 = np.array([
  [1, 1, 1],
  [4, 3, -1],
  [3, 5, 3]
], dtype='float32')

b_1 = np.array([
  [1],
  [6],
  [4]
], dtype='float32')

In [7]:
x_1 = lu_solve(A_1, b_1)
print('A solução é dada por \n x1 = {} \n x2 = {} \n x3 = {} \n'.format(x_1[0][0], 
                                                                        x_1[1][0], 
                                                                        x_1[2][0]))

A solução é dada por 
 x1 = 1.0 
 x2 = 0.5 
 x3 = -0.5 



In [8]:
A_1 @ x_1

array([[1.],
       [6.],
       [4.]])

In [9]:
# Solução com a função pronta do Numpy para comparação
np_solver_x1 = np.linalg.solve(A_1, b_1)

print('A solução é dada por \n x1 = {} \n x2 = {} \n x3 = {} \n'.format(np_solver_x1[0][0], 
                                                                        np_solver_x1[1][0], 
                                                                        np_solver_x1[2][0]))

A solução é dada por 
 x1 = 1.0 
 x2 = 0.5 
 x3 = -0.5 



In [10]:
# Solução com a função pronta do Scipy para comparação
p, l, u = scipy_lu(A_1)
np.allclose(A_1 - p @ l @ u, np.zeros((3, 3)))

True

### Exemplo 2

\begin{gather*} 
  x_1 + 4x_2 + 5x_3 = 1 \\
  6x_1 + 8x_2 - 22x_3 = 2 \\
  32x_1 + 5x_2 + 5x_3 = 3
\end{gather*}

Assim, 

\begin{gather*} 
  A_2 = \begin{bmatrix}
  1 & 4 & 5 \\ 
  6 & 8 & 22 \\ 
  32 & 5 & 5
  \end{bmatrix} 
  \qquad
  b_2 = \begin{bmatrix}
  1 \\ 
  2 \\ 
  3 
  \end{bmatrix}
\end{gather*}

In [11]:
A_2 = np.array([
  [1, 4, 5],
  [6, 8, 22],
  [32, 5, 5]
], dtype='float32')

b_2 = np.array([
  [1],
  [2],
  [3]
], dtype='float32')

In [12]:
x_2 = lu_solve(A_2, b_2)
print('A solução é dada por \n x1 = {} \n x2 = {} \n x3 = {} \n'.format(x_2[0][0], 
                                                                        x_2[1][0], 
                                                                        x_2[2][0]))

A solução é dada por 
 x1 = 0.05614973262032086 
 x2 = 0.25935828877005346 
 x3 = -0.01871657754010695 



In [13]:
A_2 @ x_2

array([[1.],
       [2.],
       [3.]])

In [14]:
# Solução com a função pronta do Numpy para comparação
np_solver_x2 = np.linalg.solve(A_2, b_2)

print('A solução é dada por \n x1 = {} \n x2 = {} \n x3 = {} \n'.format(np_solver_x2[0][0], 
                                                                        np_solver_x2[1][0], 
                                                                        np_solver_x2[2][0]))

A solução é dada por 
 x1 = 0.05614973232150078 
 x2 = 0.259358286857605 
 x3 = -0.01871657744050026 



In [15]:
# Solução com a função pronta do Scipy para comparação
p, l, u = scipy_lu(A_2)
np.allclose(A_2 - p @ l @ u, np.zeros((3, 3)))

True

### Exemplo 3

\begin{gather*} 
  4x_2 + 5x_3 = 1 \\
  6x_1 + 8x_2 - 22x_3 = 2 \\
  32x_1 + 5x_2 + 5x_3 = 3
\end{gather*}

Assim, 

\begin{gather*} 
  A_3 = \begin{bmatrix}
  0 & 4 & 5 \\ 
  6 & 8 & 22 \\ 
  32 & 5 & 5
  \end{bmatrix} 
  \qquad
  b_3 = \begin{bmatrix}
  1 \\ 
  2 \\ 
  3 
  \end{bmatrix}
\end{gather*}

In [16]:
A_3 = np.array([
  [0, 4, 5],
  [6, 8, 22],
  [32, 5, 5]
], dtype='float32')

b_3 = np.array([
  [1],
  [2],
  [3]
], dtype='float32')

> Como relembrado na **Nota** presente na descrição desta tarefa, caso não sejam tratadas, podem acontecer divisões por zero ou *near-zero*. Isso pode ser verificado abaixo quando tentamos solucionar o terceiro sistema linear. 
> 
> Tal como no método de Eliminação de *Gauss*, existem várias estratégias de pivô que podem ser utilizadas. 

In [17]:
x_3 = lu_solve(A_3, b_3)
print('A solução é dada por \n x1 = {} \n x2 = {} \n x3 = {} \n'.format(x_3[0][0], 
                                                                        x_3[1][0], 
                                                                        x_3[2][0]))

ZeroDivisionError: ignored

<!-- # Extra: Decomposição PLU

Uma das estratégias de pivô para que não faça com a que Decomposição LU falhe ao executar $a_{ii} = 0$ é utilizar a Pivotação Parcial, isto é, trocar qualquer linha que tenha um zero na diagonal pela primeira linha abaixo dela que tenha um número diferente de zero nessa coluna. 

Assim, temos que acompanhar as trocas de linha criando uma matriz de permutação $P$, de forma a executar as mesmas trocas de linha para uma matriz identidade. Isso dá uma matriz com precisamente uma entrada diferente de zero em cada linha e em cada coluna, e cada entrada diferente de zero é 1. 

A função definida abaixo realiza o processo descrito anteriormente. -->